<a id="2"></a>
# <p style="background-color:blue;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> M.TECH MAJOR PROJECT Reg BL.EN.P2DSC20005 </p>
<a id="2"></a>





<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">TABLE OF CONTENTS</p>   
    
* [1. INTRODUCTION AND IMPORTING LIBRARIES](#1)
    
* [2. LOADING DATA](#2)
    
* [3. DATA CLEANING](#3)
    
* [4. DATA PREPROCESSING](#4)   
    
* [5. DIMENSIONALITY REDUCTION](#5) 
      
* [6. EVALUATING MODELS](#6)
    
* [7. CONCLUSION](#7)
    
* [8. END  EXIT](#8)

# Instalation 
Intregating pyspark and kafka in jupyter notebook 


In [1]:
import pyspark
import findspark
findspark.init()


findspark.find()

'C:\\Users\\abhi\\.conda\\envs\\pyspark1\\lib\\site-packages\\pyspark'

<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">CREATE SPARK SESSION</p>

In [2]:
from pyspark.sql import SparkSession
sc = SparkSession.builder. \
appName("Major Project").\
master("local[1]").\
config("spark.memory.offHeap.enabled","true"). \
config("spark.memory.offHeap.size","10g").getOrCreate()


sc

<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">IMPORT LIBRARY</p>

In [3]:
import pyspark
import findspark

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession


from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline


import pandas as pd
import numpy as np


<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> DATA SET TOTAL SIZE-21 GB</p>

<a id="2"></a>
# <p style="background-color:BLUE;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">LOADING DATA SET</p>

In [4]:
df= sc.read.option("header",True).csv(r"C:\Users\abhi\Desktop\data Engineer\pysparkproject\iot23updated\iotnewdata.csv")

In [5]:
df.limit(5).toPandas()

,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_first_seen_ms,src2dst_last_seen_ms,src2dst_packets,src2dst_bytes,dst2src_first_seen_ms,dst2src_last_seen_ms,dst2src_packets,dst2src_bytes,dst2src_duration_ms,src2dst_duration_ms,Target
0,0,1,405,1537522897732,1537522897732,1,405,0,0,0,0,0,0,0
1,0,1,425,1537522897732,1537522897732,1,425,0,0,0,0,0,0,0
2,0,1,405,1537522897732,1537522897732,1,405,0,0,0,0,0,0,0
3,7005,4,972,1537523212430,1537523219435,4,972,0,0,0,0,0,7005,0
4,7006,4,972,1537523212430,1537523219436,4,972,0,0,0,0,0,7006,0


<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">OVERVIEW OF DATASET</p>

In [6]:
df.printSchema()

root
 |-- bidirectional_duration_ms: string (nullable = true)
 |-- bidirectional_packets: string (nullable = true)
 |-- bidirectional_bytes: string (nullable = true)
 |-- src2dst_first_seen_ms: string (nullable = true)
 |-- src2dst_last_seen_ms: string (nullable = true)
 |-- src2dst_packets: string (nullable = true)
 |-- src2dst_bytes: string (nullable = true)
 |-- dst2src_first_seen_ms: string (nullable = true)
 |-- dst2src_last_seen_ms: string (nullable = true)
 |-- dst2src_packets: string (nullable = true)
 |-- dst2src_bytes: string (nullable = true)
 |-- dst2src_duration_ms: string (nullable = true)
 |-- src2dst_duration_ms: string (nullable = true)
 |-- Target: string (nullable = true)



In [7]:
df.columns

['bidirectional_duration_ms',
 'bidirectional_packets',
 'bidirectional_bytes',
 'src2dst_first_seen_ms',
 'src2dst_last_seen_ms',
 'src2dst_packets',
 'src2dst_bytes',
 'dst2src_first_seen_ms',
 'dst2src_last_seen_ms',
 'dst2src_packets',
 'dst2src_bytes',
 'dst2src_duration_ms',
 'src2dst_duration_ms',
 'Target']

In [8]:
print('Number of rows: \t', df.count())
print('Number of columns: \t', len(df.columns))

Number of rows: 	 33170045
Number of columns: 	 14


In [9]:
df.groupBy('Target').count().show()

+------+--------+
|Target|   count|
+------+--------+
|     0|    2878|
|     1|33167167|
+------+--------+



In [10]:
for col in df.columns:
    print(col.ljust(20), df.filter(df[col].isNull()).count())

bidirectional_duration_ms 0
bidirectional_packets 0
bidirectional_bytes  0
src2dst_first_seen_ms 0
src2dst_last_seen_ms 0
src2dst_packets      0
src2dst_bytes        0
dst2src_first_seen_ms 0
dst2src_last_seen_ms 0
dst2src_packets      0
dst2src_bytes        0
dst2src_duration_ms  0
src2dst_duration_ms  0
Target               0


<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">DATA PREPROCESSING</p>

In [11]:
#In this code snippet conert data type each columns into integer type for feeding this data into 
# as input tovector assembler
for col in df.columns:
    df = df.withColumn(col, df[col].cast(IntegerType()))

In [12]:
df.printSchema()

root
 |-- bidirectional_duration_ms: integer (nullable = true)
 |-- bidirectional_packets: integer (nullable = true)
 |-- bidirectional_bytes: integer (nullable = true)
 |-- src2dst_first_seen_ms: integer (nullable = true)
 |-- src2dst_last_seen_ms: integer (nullable = true)
 |-- src2dst_packets: integer (nullable = true)
 |-- src2dst_bytes: integer (nullable = true)
 |-- dst2src_first_seen_ms: integer (nullable = true)
 |-- dst2src_last_seen_ms: integer (nullable = true)
 |-- dst2src_packets: integer (nullable = true)
 |-- dst2src_bytes: integer (nullable = true)
 |-- dst2src_duration_ms: integer (nullable = true)
 |-- src2dst_duration_ms: integer (nullable = true)
 |-- Target: integer (nullable = true)



<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">CREATING FEATURE VECTOR USING VECTOR ASSEMBLER </p>

Preparing Dataset for Machine Learning

In [13]:
featureCols= ['bidirectional_duration_ms','bidirectional_packets','bidirectional_bytes',
              'src2dst_packets','src2dst_bytes','dst2src_packets','dst2src_bytes',
              'dst2src_duration_ms','src2dst_duration_ms']

In [14]:
featureCols=

SyntaxError: invalid syntax (1266613787.py, line 1)

In [ ]:
len(featureCols)

In [ ]:
# put features into a feature vector column
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


assembler = VectorAssembler(inputCols=featureCols, outputCol="features") 
assembled_df = assembler.transform(df)

In [ ]:
assembled_df.limit(2).toPandas()

In [ ]:
assembled_df.columns

In [ ]:
assembled_df.select("features").show()

<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">1.CREATING NEW DATAFRAME(model_df) FOR ML MODEL</p>

In [ ]:
assembled_df.columns

In [ ]:
# Inspect the result
assembled_df.select("Target", "features").show(2, truncate=False)

In [ ]:
model_df = assembled_df.select(['Target','features'])

In [ ]:
# Inspect the result
model_df.select("Target", "features").show(2, truncate=False)

<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Random Forest Classifier</p>

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
train_data, test_data,validation_data = model_df.randomSplit([0.2, 0.2, 0.6])

In [ ]:
rf_classifier=RandomForestClassifier(labelCol='Target',numTrees=50).fit(train_data)

In [ ]:
rf_predictions=rf_classifier.transform(test_data)

In [ ]:
rf_predictions.show()

In [ ]:
rf_predictions.groupBy('prediction').count().show()

In [ ]:
rf_predictions.select(['probability','Target','prediction']).show(10,False)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='Target',metricName='accuracy').evaluate(rf_predictions)

In [ ]:
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

In [ ]:
print(rf_accuracy)

In [ ]:
rf_precision=MulticlassClassificationEvaluator(labelCol='Target',metricName='weightedPrecision').evaluate(rf_predictions)

In [ ]:
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

In [ ]:
rf_precision

In [ ]:
rf_auc=BinaryClassificationEvaluator(labelCol='Target').evaluate(rf_predictions)

In [ ]:
print(rf_auc)

In [ ]:
rf_classifier.featureImportances

In [ ]:
#END 15-2-2023


In [ ]:
m

In [ ]:
pwd

In [ ]:
rf_classifier.save('C:/Users/abhi/Desktop/data Engineer/pysparkproject/mrfmodel')

In [ ]:
 pwd

In [ ]:
from pyspark.ml.classification import RandomForestClassificationModel

In [ ]:
rf=RandomForestClassificationModel.load('C:/Users/abhi/Desktop/data Engineer/pysparkproject/mrfmodel')

In [ ]:
model_preditions=rf.transform(test_data)

In [ ]:
model_preditions.show()



<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">STANDARD SCALER</p>

In [ ]:
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaled_df = standardScaler.fit(assembled_df).transform(assembled_df)

In [ ]:
scaled_df.limit(1).toPandas()

In [ ]:
# Inspect the result
scaled_df.select("Target", "features_scaled").show(2, truncate=False)

<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">CREATING NEW DATAFRAME(model_df) FOR ML MODEL</p>

In [ ]:
model_df = scaled_df.select(['Target','features_scaled'])

In [ ]:
model_df.groupBy('Target').count().show()


<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">SPLIT DATASET INTO Train_data,Test_data AND Validation SET</p>

In [ ]:
train_data, test_data,validation_data = model_df.randomSplit([0.2, 0.2, 0.6])

In [ ]:
print('Train Shape: ({}, {})'.format(train_data.count(), len(train_data.columns)))

In [ ]:
print('Test Shape: ({}, {})'.format(test_data.count(), len(test_data.columns)))

In [ ]:
print('validity: ({}, {})'.format(validation_data.count(), len(validation_data.columns)))


<a id="4"></a>
# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">model</p>

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rf_classifier=RandomForestClassifier(labelCol='Target',numTrees=50).fit(train_data)

In [ ]:
rf_predictions=rf_classifier.transform(test_df)

In [ ]:
rf_predictions.show()



<a id="4"></a>
# <p style="background-color:#682F2F;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">CREATING CSV FILE FROM OF Train_data.csv,test_data.csv and validation.csv</p>





In [ ]:
for col in train_data.columns:
    df1 = train_data.withColumn(col,train_data[col].cast(StringType()))

In [ ]:
df2.write.options(header='True', delimiter=',') \
 .csv("/tmp/spark_output/zipcodes")

In [ ]:
df1.coalesce(1).write.options(header='True').mode('overwrite').csv("C:/Users/abhi/Desktop/data Engineer/pysparkproject/train121")

In [ ]:
for col in test_data.columns:
    df2 = test_data.withColumn(col, test_data[col].cast(StringType()))

In [ ]:
df2.coalesce(1).write.options(header='True').mode('overwrite').csv("C:/Users/abhi/Desktop/data Engineer/pysparkproject/test")

In [ ]:
for col in validation_data.columns:
    df3 = validation_data.withColumn(col, validation_data[col].cast(StringType()))

In [ ]:
df3.coalesce(1).write.options(header='True').mode('overwrite').csv("C:/Users/abhi/Desktop/data Engineer/pysparkproject/validation121")

In [ ]:
pwd

<a id="2"></a>
# <p style="background-color:lightpink;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">TRAINING AND MODEL FIT </p>

#work on progress